In [10]:
import numpy as np

class NeuralNetwork:
    INPUT_NEURONS = 9  # Количество нейронов во входном слое
    HIDDEN_LAYER1_NEURONS = 2  # Количество нейронов в первом скрытом слое
    HIDDEN_LAYER2_NEURONS = 3  # Количество нейронов во втором скрытом слое
    OUTPUT_NEURONS = 10  # Количество нейронов в выходном слое (цифры 0-9)
    LEARNING_RATE = 0.7  # Скорость обучения
    ITERATIONS = 100000  # Количество итераций обучения

    def __init__(self):
        # Инициализация слоев и весов
        self.inputs = np.zeros(self.INPUT_NEURONS)  # Входные значения
        self.hiddenLayer1 = np.zeros(self.HIDDEN_LAYER1_NEURONS)  # Первый скрытый слой
        self.hiddenLayer2 = np.zeros(self.HIDDEN_LAYER2_NEURONS)  # Второй скрытый слой
        self.outputs = np.zeros(self.OUTPUT_NEURONS)  # Выходной слой

        # Матрицы весов: случайные значения от -0.5 до 0.5
        self.weightsInputToHidden1 = np.random.rand(self.INPUT_NEURONS, self.HIDDEN_LAYER1_NEURONS) - 0.5
        self.weightsHidden1ToHidden2 = np.random.rand(self.HIDDEN_LAYER1_NEURONS, self.HIDDEN_LAYER2_NEURONS) - 0.5
        self.weightsHidden2ToOutput = np.random.rand(self.HIDDEN_LAYER2_NEURONS, self.OUTPUT_NEURONS) - 0.5

    def sigmoid(self, x):
        # Активационная функция (сигмоида)
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        # Производная сигмоиды для вычисления градиента
        return x * (1 - x)

    def forward_pass(self, inputs):
        # Прямой проход данных через сеть (вычисление выходов)
        self.inputs = inputs

        # Расчёт выходов первого скрытого слоя
        self.hiddenLayer1 = self.sigmoid(np.dot(self.inputs, self.weightsInputToHidden1))

        # Расчёт выходов второго скрытого слоя
        self.hiddenLayer2 = self.sigmoid(np.dot(self.hiddenLayer1, self.weightsHidden1ToHidden2))

        # Расчёт выходов выходного слоя
        self.outputs = self.sigmoid(np.dot(self.hiddenLayer2, self.weightsHidden2ToOutput))

        return self.outputs

    def backpropagation(self, expected):
        # Обратное распространение ошибки для корректировки весов

        # Ошибка выходного слоя
        output_errors = (expected - self.outputs) * self.sigmoid_derivative(self.outputs)

        # Ошибка второго скрытого слоя
        hiddenLayer2_errors = np.dot(output_errors, self.weightsHidden2ToOutput.T) * self.sigmoid_derivative(self.hiddenLayer2)

        # Ошибка первого скрытого слоя
        hiddenLayer1_errors = np.dot(hiddenLayer2_errors, self.weightsHidden1ToHidden2.T) * self.sigmoid_derivative(self.hiddenLayer1)

        # Корректировка весов между слоями
        self.weightsHidden2ToOutput += self.LEARNING_RATE * np.outer(self.hiddenLayer2, output_errors)
        self.weightsHidden1ToHidden2 += self.LEARNING_RATE * np.outer(self.hiddenLayer1, hiddenLayer2_errors)
        self.weightsInputToHidden1 += self.LEARNING_RATE * np.outer(self.inputs, hiddenLayer1_errors)

    def train(self, training_inputs, training_outputs):
        # Обучение сети с использованием предоставленных данных
        for _ in range(self.ITERATIONS):
            for inputs, expected in zip(training_inputs, training_outputs):
                self.forward_pass(inputs)  # Прямой проход
                self.backpropagation(expected)  # Корректировка весов

    def predict(self, input):
        # Предсказание результата для новых входных данных
        outputs = self.forward_pass(input)
        return np.argmax(outputs)  # Индекс максимального значения

    def print_weights(self):
        # Вывод текущих весовых коэффициентов для всех слоёв
        print("Weights from Input to Hidden Layer 1:")
        print(self.weightsInputToHidden1)

        print("\nWeights from Hidden Layer 1 to Hidden Layer 2:")
        print(self.weightsHidden1ToHidden2)

        print("\nWeights from Hidden Layer 2 to Output:")
        print(self.weightsHidden2ToOutput)

# Инициализация нейронной сети и обучение
nn = NeuralNetwork()

# Набор данных для обучения (входы и ожидаемые выходы)
training_inputs = np.array([
    [1, 1, 1, 1, 0, 1, 1, 1, 1],  # Цифра 0
    [0, 0, 1, 0, 0, 1, 0, 0, 1],  # Цифра 1
    [1, 0, 1, 1, 1, 1, 0, 0, 1],  # Цифра 4
    [1, 1, 1, 0, 0, 1, 0, 0, 1]   # Цифра 7
])

training_outputs = np.array([
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # Цифра 0
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],  # Цифра 1
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],  # Цифра 4
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]   # Цифра 7
])

# Обучение сети
nn.train(training_inputs, training_outputs)

# Вывод начальных весов
print("Initial Weights:")
nn.print_weights()

# Тестирование сети на новых данных
test_input = np.array([0, 0, 1, 0, 0, 1, 0, 0, 0])  # Пример теста цифры 1
prediction = nn.predict(test_input)
print("\nPredicted digit:", prediction)

# Вывод финальных весов
print("\nFinal Weights:")
nn.print_weights()


Initial Weights:
Weights from Input to Hidden Layer 1:
[[-4.5497589  -0.43191305]
 [-0.37483429  2.63717001]
 [ 0.44933618  0.24885991]
 [-0.98848348 -5.45376536]
 [-4.10658773 -3.65256251]
 [ 0.88514657  1.14534416]
 [ 3.75703228 -2.04527213]
 [ 3.49666966 -1.64471118]
 [ 0.57414101  0.39377363]]

Weights from Hidden Layer 1 to Hidden Layer 2:
[[-7.83120241  3.10915854  5.11614363]
 [ 2.69639055 -7.68385228  5.07620229]]

Weights from Hidden Layer 2 to Output:
[[-15.55103706 -11.98246206  -3.1393338   -3.25969994  13.48887431
   -3.25008318  -3.50296659  11.05069005  -3.57581855  -2.86789839]
 [ 10.9352301  -11.4392451   -3.30008646  -3.18781021  12.82612106
   -3.15792909  -2.97751237 -16.02336285  -2.99811959  -3.42552722]
 [ -5.36573954   5.41761718  -5.92148324  -5.92128289 -17.31364973
   -5.9287696   -5.91858553  -5.02286003  -5.9035279   -5.94961737]]

Predicted digit: 1

Final Weights:
Weights from Input to Hidden Layer 1:
[[-4.5497589  -0.43191305]
 [-0.37483429  2.63717001]
